In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from mss import mss

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import pyautogui
import time
#env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Input Extraction(Screen Extraction)
from PIL import ImageGrab
import cv2
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [3]:
cv2.destroyAllWindows()

## Templates

In [4]:

white_template = cv2.imread('templates/white_temp.png',0) 
blue_template = cv2.imread('templates/blue_temp.png',0)
orange_template = cv2.imread('templates/orange_temp.png',0)
green_template = cv2.imread('templates/green_temp.png',0)
cv2.imshow('image',blue_template)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rewards skeleton

## Initial block count

In [5]:
#one scrren
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
sct_img = np.array(sct.grab(bounding_box))
gray = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
#cv2.imshow('image',gray)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
w,h = blue_template.shape[::-1]  #here
result = cv2.matchTemplate(gray,blue_template, cv2.TM_CCOEFF_NORMED) #here 
f = set()
loc = np.where(result >=0.9)
for pt in zip(*loc[::-1]):
    cv2.rectangle(sct_img,pt,(pt[0] + w, pt[1] + h),(0,255,0),2)
    sensitivity = 100
    f.add((round(pt[0]/sensitivity), round(pt[1]/sensitivity)))
    
found_count = len(f)    
print("amount of blocks", found_count)
cv2.imshow("result", sct_img)
#cv2.imshow('screen', np.array(sct_img))
cv2.waitKey(0)
cv2.destroyAllWindows()
initial_block_value = found_count
print(initial_block_value)

amount of blocks 21
21


In [6]:
#screen method 1
#img = ImageGrab.grab(bbox=(200,630,850,600)) #bbox specifies specific region (bbox= x,y,width,height *starts top-left)
#img_np = np.array(img) #this is the array obtained from conversion
#frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
#cv2.imshow("test", frame)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
   

#one scrren
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()

In [8]:
#while True:
#7 outputs(mouse positions)
sct_img = sct.grab(bounding_box)
cv2.imshow('screen', np.array(sct_img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def transform_screen_data2(screen):
        screen = np.ascontiguousarray(screen, dtype=np.float32)/255
        screen = torch.from_numpy(screen).float()
        screen = transforms.ToPILImage()(screen).convert("RGB")
        resize = T.Compose([
             T.Resize((600,850))
            ,T.ToTensor()
        ])
        return resize(screen).unsqueeze(0).to(device = "cuda") # add a batch dimension (BCHW)

In [7]:
def transform_screen_data1(screen):
        screen = np.ascontiguousarray(screen, dtype=np.float32)/255
        screen = torch.from_numpy(screen).float()
        return screen # add a batch dimension (BCHW)

In [8]:
from torchvision import transforms
sct = mss()
sct_img =(sct.grab(bounding_box))
im = transforms.ToPILImage()(np.array(sct_img)).convert("RGB")
#display(im)
print(im)
print(im.size)

<PIL.Image.Image image mode=RGB size=850x600 at 0x13C210D03C8>
(850, 600)


In [59]:
from torchvision import transforms
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
sct_img =(sct.grab(bounding_box))
sct_img = np.array(sct_img)
sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
im = transform_screen_data1(sct_img)
sct = mss()
sct_img =(sct.grab(bounding_box))
sct_img = np.array(sct_img)
sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
im1 = transform_screen_data1(sct_img)
result = im1 - im
print(result.shape)
#print(result.squeeze().unsqueeze(dim=0))
print(result.squeeze().unsqueeze(dim=0).shape)
#cv2.imshow("res",np.array(result))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

torch.Size([600, 850])
torch.Size([1, 600, 850])


In [18]:
            sct_img =(sct.grab(bounding_box))
            second_example2 = transform_screen_data1(sct_img)
            sct_img =(sct.grab(bounding_box))
            second_example3 = transform_screen_data1(sct_img)
            
            screen_return = second_example3-second_example2 #(Current minus previous)
            print(screen_return.shape[0])
            print(screen_return)
            cv2.imshow("res",np.array(screen_return))
            cv2.waitKey(0)
            cv2.destroyAllWindows()

600
tensor([[[-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         [-0.0039, -0.0039, -0.0078,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        ...,

        [[ 0.0039,  0.0039,  0.0039,  0.0000],
         [ 0.0039,  0.0039,  0.0039,  0.0

In [14]:


#screen method 2(Current)
#bounding_box = {'top': 730, 'left': 600, 'width': 850, 'height': 80} #top left corner of screen
bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
sct = mss()
while True:
    sct_img =(sct.grab(bounding_box))
    sct_img = np.array(sct_img)
    first_example = transform_screen_data1(sct_img) #Transform screen(current screen)/previous
    sct_img =(sct.grab(bounding_box))
    sct_img = np.array(sct_img)
    second_example = transform_screen_data1(sct_img) #Transform screen(current screen)/previous
    example = second_example -first_example 
    #print(example)
    #state = example
    cv2.imshow('screen',(np.array(example)))
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break

# Actions Mapping

In [9]:
#10 outputs
def take_actions(action):
    if action == 0:
        pyautogui.moveTo(640, 400, duration = 0.07) 
    elif action == 1:
        pyautogui.moveTo(740, 400, duration = 0.07) 
    elif action == 2:
        pyautogui.moveTo(840, 400, duration = 0.07)
    elif action ==3:
        pyautogui.moveTo(940, 400, duration = 0.07) 
    elif action ==4:
        pyautogui.moveTo(1040, 400, duration = 0.07)
    elif action ==5:
        pyautogui.moveTo(1140, 400, duration = 0.07)
    elif action ==6:
        pyautogui.moveTo(1240, 400, duration = 0.07)
    elif action == 7:
        pyautogui.moveTo(1340, 400, duration = 0.07) 
    elif action == 8:
        pyautogui.moveTo(1440, 400, duration = 0.07)
    elif action == 9:
        pyautogui.moveTo(1500, 400, duration = 0.07)
    else:
        print("Not allowed")

# Experience

In [10]:
from collections import namedtuple
Xp = namedtuple('Experience',
                        ('state', 'action', 'next_state', 'reward'))
Xp_points = Xp(5,6,7,8)
Xp_points

Experience(state=5, action=6, next_state=7, reward=8)

# Network Skeleton

In [60]:
#--------------------------------------DQN(Fully connected)-----------------------------------------#
class DQN (nn.Module):
    def __init__(self,img_height, img_width):
        super().__init__()
        self.input_layer = nn.Linear(in_features = img_height*img_width,out_features=850)
        self.layer2      = nn.Linear(in_features = 850, out_features = 200)
        self.layer3      = nn.Linear(in_features = 200, out_features = 200)
        self.layer4      = nn.Linear(in_features = 200, out_features = 200)
        self.layer5      = nn.Linear(in_features = 200, out_features = 132)
        self.layer6      = nn.Linear(in_features = 132, out_features = 100)
        self.output      = nn.Linear(in_features = 100, out_features = 10)
#--------------------forward pass---------------------------------------------------------------------#
    def forward(self,t):
        t = t.flatten(start_dim=1)
        #t = t.reshape(-1)
        #t = t.flatten(start_dim = 0)
        #print("tensor_input_size", t.size())
        t = F.relu(self.input_layer(t))
        t = F.relu(self.layer2(t))
        t = F.relu(self.layer3(t))
        t = F.relu(self.layer4(t))
        t = F.relu(self.layer5(t))
        t = F.relu(self.layer6(t))
        t = self.output(t)
        #print(t.item())
        #print(t)
        return t

## Learning rate and network setup

In [12]:
#lr = 0.01
#policy_net = DQN(600,850).to(device = "cpu")
#target_net = DQN(600,850).to(device = "cpu")
#target_net.load_state_dict(policy_net.state_dict()) #set weights to be the same
#target_net.eval() #not in training mode
#optimizer = optim.Adam(params = policy_net.parameters(), lr=lr)

# Q values setup (fix this up abit)

In [81]:
class Q_Values():
    device = "cpu"
    
    @staticmethod
    def get_current(policy_net, states, actions):
        print("action shape", actions.shape)
        actions = actions.squeeze()
        print("action shape", actions.shape)
        #print("states shape",states.shape)
        #return policy_net(states).gather(index=actions.unsqueeze(-1))
        return policy_net(states).gather(dim=1, index=actions.unsqueeze(1))
    
    @staticmethod        
    def get_next(target_net, next_states):
        #print("nx",next_states)
        #print("shape", next_states.shape[0])
        final_state_locations = next_states.flatten(start_dim=1).max(dim=1)[0].eq(0).type(torch.bool)
        print("ft",final_state_locations)
        non_final_state_locations = (final_state_locations == False)
        print("nf",non_final_state_locations)
        non_final_states = next_states[non_final_state_locations]
        batch_size = next_states.shape[0]
        values = torch.zeros(batch_size).to(Q_Values.device)
        #values = torch.zeros(2).to(Q_Values.device)
        values[non_final_state_locations] = target_net(non_final_states).max(dim=1)[0].detach()
        return values

# Tensor Extraction

In [48]:
def extract_tensors(experiences):
    batch = Xp(*zip(*experiences))
    t1 = torch.stack(batch.state) #stack
    print("t1",t1)
    t2 = torch.stack(batch.action)
    print("t2",t2)
    t3 = torch.stack(batch.reward)
    print("t3",t3)
    t4 = torch.stack(batch.next_state)
    print("t4",t4)
    
    
    return t1,t2,t3,t4

# Replay Memory

In [15]:
#-----------------------------DEJAVU----------------------------------------------------------#
class ReplayMemory():
    def __init__(self,capacity):   
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            #overwrite old memory
            self.memory[self.push_count%self.capacity] = experience
        self.push_count+=1
    #number of returned memories will be equal to batch size
    def sample(self, batch_size):
        return rand.sample(self.memory,batch_size)
    #sample must be equal to batch size
    def can_provide_sample(self, batch_size):
        return len(self.memory)>=batch_size
    

# Epsilon Decay Rate

In [16]:
class Greed():
    def __init__(self,start,end,decay):
        self.start = start
        self.end = end
        self.decay = decay
    def get_exploration_rate(self,current_step):
        return self.end + (self.start - self.end)*math.exp(-1. * current_step*self.decay)

In [67]:
memory = ReplayMemory(100000)
gamma = 0.7 #affects network
import random as rand

In [68]:
w,h = blue_template.shape[::-1]  #set template shape


In [69]:
lr = 0.01
policy_net = DQN(600,850).to(device = "cpu")
target_net = DQN(600,850).to(device = "cpu")
target_net.load_state_dict(policy_net.state_dict()) #set weights to be the same
target_net.eval() #not in training mode
optimizer = optim.Adam(params = policy_net.parameters(), lr=lr)

In [21]:
def plot(values, moving_avg_period):
    plt.figure(2)
    plt.clf()        
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(values)
    
    moving_avg = get_moving_average(moving_avg_period, values)
    plt.plot(moving_avg)    
    plt.pause(0.001)
    print("Episode", len(values), "\n", \
          moving_avg_period, "episode moving avg:", moving_avg[-1])
    if is_ipython: display.clear_output(wait=True)

def get_moving_average(period, values):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1) \
            .mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()

# Main

In [82]:
#def launch():
exp_rate = 1
episode = 0
max_rate = 1
min_rate = 0.01
exp_decay = 0.001
reward_count = 0
for n in range (200):
    w,h = blue_template.shape[::-1]  #set template shape
    count = 0
    current_blocks = initial_block_value
    past_blocks = initial_block_value
    reward = 0
    time.sleep(7) #delay
    bounding_box = {'top': 200, 'left': 630, 'width': 850, 'height': 600} #top left corner of screen
    #bounding_box = {'top': 200, 'left': 600, 'width': 850, 'height': 600} 
    sct = mss()
    sct_img =(sct.grab(bounding_box))
    sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
    first_example = transform_screen_data1(sct_img) #Transform screen(current screen)/previous
    sct_img =(sct.grab(bounding_box))
    sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
    second_example = transform_screen_data1(sct_img) #Transform screen(current screen)/previous
    example = second_example - first_example
    state = example
    state = state.squeeze().unsqueeze(dim=0)
    print("initial state", state.shape)
    done = False
    while done is not True:
        #10 outputs(mouse positions)
        init_random = np.random.uniform(0,1)
        past_blocks = current_blocks
        count+=1
        print("click")
        x,y = pyautogui.position()
        pyautogui.click(x, y)
###############################################ACTION########################################################
        #epsilon greed
        exp_thresh = rand.uniform(0,1)
        if exp_thresh < exp_rate:
             with torch.no_grad():#need to review this
                action = policy_net(state).argmax(dim=1).to(device ="cpu")#argmat was -1
                print("n",action.shape)
        else:
            rndm = rand.randrange(0,9)
            action = torch.tensor([rndm]).to(device = "cpu")
            print("rand_value", action)
        take_actions(action)
#############################################REWARD########################################################### 
        sct_img =(sct.grab(bounding_box))
        gray = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY) #block detection
        f = set()
        result = cv2.matchTemplate(gray,blue_template, cv2.TM_CCOEFF_NORMED) #match template per frame
        loc = np.where(result >=0.9)
        for pt in zip(*loc[::-1]):
            sensitivity = 100
            f.add((round(pt[0]/sensitivity), round(pt[1]/sensitivity)))
        found_count = len(f) # reward factor
        current_blocks = found_count
        #reward feed to network(onl give reard if resulting action is not a black screen)
        if np.average(gray) > 20:
            if(current_blocks < past_blocks):
                print("current_blocks", current_blocks,"past_blocks",past_blocks )
                reward =(past_blocks - current_blocks)
                reward_count+=reward
                print("sub",reward)
                reward = torch.FloatTensor([reward], device = "cpu")
                print("reward_tensor0",reward)
                
            else:
                print("current_blocks1", current_blocks,"past_blocks1",past_blocks )
                reward = -1
                reward_count+=reward
                reward = torch.FloatTensor([reward], device = "cpu")
                print("reward_tensor1",reward)
        else:
            done = True
            #print("its dark")
            reward = -4
            reward = torch.FloatTensor([reward], device = "cpu")
        
        #from that action i got this state
        #sct_img =(sct.grab(bounding_box))
        #gray1 = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY) #block detection
#########################################NEXT_STATE############################################################        
        # if resulting action is a black screen
        if(np.average(gray) < 20):
            screen_return = torch.zeros_like(state)
            done = True
        else:
            sct_img =(sct.grab(bounding_box))
            sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
            second_example2 = transform_screen_data1(sct_img)
            sct_img =(sct.grab(bounding_box))
            sct_img = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2GRAY)
            second_example3 = transform_screen_data1(sct_img)
            
            screen_return = second_example3-second_example2 #(Current minus previous)
        next_state = screen_return
        next_state = next_state.squeeze().unsqueeze(dim=0)
        print("nxt_sts",next_state.shape)
        #print("tensor_input_size", next_state.size())
        memory.push(Xp(state,action,next_state,reward))
        #print(Xp(state,action,next_state,reward))
        state = next_state
######################################EXPERIENCE AND Q VALUES##################################################
        if memory.can_provide_sample(50):
            Xps = memory.sample(50)
            print("sampling")
            states, actions,rewards,next_states = extract_tensors(Xps)
            current_q_values = Q_Values.get_current(policy_net,states,actions)
            next_q_values = Q_Values.get_next(target_net,next_states)
            target_q_values = (next_q_values *(gamma)) + rewards
            loss = F.mse_loss(current_q_values,target_q_values.unsqueeze(1))
            optimizer.zero_grad() #zero out gradients to avoid turning towarda a certain direction
            loss.backward()
            optimizer.step() #back propagate
        if np.average(gray) < 20:
            done = True
            #plot(reward_count, 100)
            print("Exiting loop...")
            #break
        #cv2.imshow('screen', np.array(sct_img))
        #if (cv2.waitKey(1) & 0xFF) == ord('q'):
            #cv2.destroyAllWindows()
            #break
    if n % 100 == 0:
        print("updated policy")
        target_net.load_state_dict(policy_net.state_dict())
    exp_rate = min_rate + (max_rate - min_rate)*np.exp(-exp_decay * n)

initial state torch.Size([1, 600, 850])
click
n torch.Size([1])
current_blocks1 21 past_blocks1 21
reward_tensor1 tensor([-1.])
nxt_sts torch.Size([1, 600, 850])
sampling
t1 tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


c:\users\scowt\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:114: UserWarning: Using a target size (torch.Size([50, 1, 50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


click
n torch.Size([1])
nxt_sts torch.Size([1, 600, 850])
sampling
t1 tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0.,

KeyboardInterrupt: 

In [ ]:
#launch()